In [28]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math

In [69]:
with open("/home/peter/Downloads/AdventOfCode/day16Input.txt") as fh:
    lines = fh.readlines()
#with open("/home/peter/Downloads/AdventOfCode/day15Test.txt") as fh:
#    lines = fh.readlines()


In [70]:
data = lines[0].strip()
data

'005532447836402684AC7AB3801A800021F0961146B1007A1147C89440294D005C12D2A7BC992D3F4E50C72CDF29EECFD0ACD5CC016962099194002CE31C5D3005F401296CAF4B656A46B2DE5588015C913D8653A3A001B9C3C93D7AC672F4FF78C136532E6E0007FCDFA975A3004B002E69EC4FD2D32CDF3FFDDAF01C91FCA7B41700263818025A00B48DEF3DFB89D26C3281A200F4C5AF57582527BC1890042DE00B4B324DBA4FAFCE473EF7CC0802B59DA28580212B3BD99A78C8004EC300761DC128EE40086C4F8E50F0C01882D0FE29900A01C01C2C96F38FCBB3E18C96F38FCBB3E1BCC57E2AA0154EDEC45096712A64A2520C6401A9E80213D98562653D98562612A06C0143CB03C529B5D9FD87CBA64F88CA439EC5BB299718023800D3CE7A935F9EA884F5EFAE9E10079125AF39E80212330F93EC7DAD7A9D5C4002A24A806A0062019B6600730173640575A0147C60070011FCA005000F7080385800CBEE006800A30C023520077A401840004BAC00D7A001FB31AAD10CC016923DA00686769E019DA780D0022394854167C2A56FB75200D33801F696D5B922F98B68B64E02460054CAE900949401BB80021D0562344E00042A16C6B8253000600B78020200E44386B068401E8391661C4E14B804D3B6B27CFE98E73BCF55B65762C402768803F09620419100661EC2A8CE0008741

In [23]:
binData = bin(int(data,16))[2:]

In [177]:
class decoder:
    def __init__(self,data,debug=False,debug2=False):
        self.data = format(int(data,16),f"0{len(data)*4}b")
        #self.data = bin(int(data,16))[2:]
        self.index = 0
        self.versionSum = 0
        self.debug = debug
        self.debug2 = debug2
        if self.debug == True:print(self.data)
    def getBits(self,nBits):
        r = self.data[self.index:self.index+nBits]
        self.index += nBits
        return int(r,2)
    def getPacket(self,inSubPacket):
        startIndex = self.index
        version = self.getBits(3)
        if self.debug == True:print("vers is ",version)
        self.versionSum += version
        typeID = self.getBits(3)
        if self.debug == True:print("type id ",typeID)
        if typeID == 4:
            # literal
            if self.debug == True:print("literal")
            keepGoing = 1
            val=0
            while keepGoing:
                keepGoing = self.getBits(1)
                val = (val<<4) + self.getBits(4)
        else:
            # operator
            if self.debug == True:print("operator")
            lengthTypeId = self.getBits(1)
            if lengthTypeId == 0:
                if self.debug == True:print("  typeId 0")
                lengthInBits = self.getBits(15)
                end = self.index + lengthInBits
                if self.debug == True:print("  lenght is ",lengthInBits)
                vals = []
                while(self.index < end):
                    vals.append(self.getPacket(True))
                # todo: we would interpret the packet
                # skip for now:
            else:
                if self.debug == True:print("  typeId 1")
                numberOfSubPackets = self.getBits(11)
                if self.debug == True:print("  ",numberOfSubPackets," packets")
                vals = []
                for i in range(numberOfSubPackets):
                    vals.append(self.getPacket(True))
            if self.debug2 == True:print("  vals are ",vals)
            if typeID == 0:
                if self.debug2 == True:print("  sum")
                val = np.sum(vals)
                # sum
            elif typeID == 1:
                if self.debug2 == True:print("  prod")
                # product
                val = np.prod(vals)
            elif typeID == 2:
                if self.debug2 == True:print("  min")
                # min
                val = np.min(vals)
            elif typeID == 3:
                if self.debug2 == True:print("  max")
                # max
                val = np.max(vals)
            elif typeID == 5:
                if self.debug2 == True:print("  gt")
                # greaterthen
                val = 1 if vals[0] > vals[1] else 0
            elif typeID == 6:
                if self.debug2 == True:print("  lt")
                # lessthen
                val = 1 if vals[0] < vals[1] else 0

            elif typeID == 7:
                if self.debug2 == True:print("  eq")
                # ==
                val = 1 if vals[0] == vals[1] else 0
                
            if self.debug2 == True:print("  val:",val)
        if not inSubPacket:
            if self.debug == True:print("ind was ",self.index)
            self.index = math.ceil(self.index/4)*4
            if self.debug == True:print("updated to ",self.index)
        if self.debug == True:print("packet was ",self.data[startIndex:self.index])
        return val
    def doDecode(self):
        #while(self.index < len(self.data)):
            #if self.debug == True:print("index is ",self.index)
        return self.getPacket(False)

In [178]:
def tryFoo(val,debug=False):
    foo = decoder(val,debug)
    print("res is ",foo.doDecode())


#foo = decoder("8A004A801A8002F478")
foo = decoder("D2FE28")
print("res is :",foo.doDecode())
print("expected 6")
print(foo.versionSum)

foo = decoder("38006F45291200")
print("res is :",foo.doDecode())
print("expected x")
print(foo.versionSum)

foo = decoder("8A004A801A8002F478",True)
print("res is :",foo.doDecode())
print("expected 16")
print(foo.versionSum)
foo = decoder("620080001611562C8802118E34")
print("res is :",foo.doDecode())
print("expected 12")
print(foo.versionSum)
foo = decoder("C0015000016115A2E0802F182340")
print("res is :",foo.doDecode())
print("expected 23")
print(foo.versionSum)

foo = decoder("A0016C880162017C3686B18A3D4780")
print("res is :",foo.doDecode())
print("expected 31")
print(foo.versionSum)






res is : 2021
expected 6
6
res is : 1
expected x
9
100010100000000001001010100000000001101010000000000000101111010001111000
vers is  4
type id  2
operator
  typeId 1
   1  packets
vers is  1
type id  2
operator
  typeId 1
   1  packets
vers is  5
type id  2
operator
  typeId 0
  lenght is  11
vers is  6
type id  4
literal
packet was  11010001111
packet was  101010000000000000101111010001111
packet was  001010100000000001101010000000000000101111010001111
ind was  69
updated to  72
packet was  100010100000000001001010100000000001101010000000000000101111010001111000
res is : 15
expected 16
16
res is : 46
expected 12
12
res is : 46
expected 23
23
res is : 54
expected 31
31


In [179]:
tryFoo("C200B40A82")
tryFoo("04005AC33890")
tryFoo("880086C3E88112")
tryFoo("CE00C43D881120")
tryFoo("D8005AC2A8F0")
tryFoo("F600BC2D8F")
tryFoo("9C005AC2F8F0")
tryFoo("9C0141080250320F1802104A08")




foo = decoder(data,False,True)
print(foo.doDecode())

res is  3
res is  54
res is  7
res is  9
res is  1
res is  0
res is  0
res is  1
  vals are  [4, 4, 13]
  sum
  val: 21
  vals are  [4, 15, 9]
  sum
  val: 28
  vals are  [21, 28]
  lt
  val: 1
  vals are  [20955, 1]
  prod
  val: 20955
  vals are  [6442729, 429012323437]
  gt
  val: 0
  vals are  [0, 21160133946]
  prod
  val: 0
  vals are  [100, 147]
  prod
  val: 14700
  vals are  [12, 11]
  min
  val: 11
  vals are  [1691, 1691]
  lt
  val: 0
  vals are  [0, 1387418]
  prod
  val: 0
  vals are  [23, 6, 211]
  prod
  val: 29118
  vals are  [7, 241254, 7667148, 181, 118]
  sum
  val: 7908708
  vals are  [779387]
  sum
  val: 779387
  vals are  [243524268, 50733999875]
  lt
  val: 1
  vals are  [1, 2040760]
  prod
  val: 2040760
  vals are  [14]
  max
  val: 14
  vals are  [393461449, 60]
  lt
  val: 0
  vals are  [0, 65]
  prod
  val: 0
  vals are  [550750, 2, 15238]
  sum
  val: 565990
  vals are  [108, 190]
  eq
  val: 0
  vals are  [0, 251]
  prod
  val: 0
  vals are  [341868, 230

In [180]:
print(foo.versionSum)

981


In [110]:
if True: print ("hi")

hi


In [181]:
len(bin(299227024091))

41